In [514]:
import numpy as np 
import pandas as pd
np.random.seed(42)

In [724]:
dados = pd.read_csv("dados_ajustados.csv", decimal= ",", delimiter =";")
dados = dados[["SPXI", "IBOV"]]
retornos = dados.pct_change().dropna()
cov = np.array(retornos.cov())
e_r = np.array(retornos.mean()).reshape(2, 1)
rp = 0.00178


In [610]:
def chromossome(n):
    x = np.random.rand(n)
    return x/sum(x)

In [611]:
w = chromossome(2)
w

array([0.15647897, 0.84352103])

In [703]:
def fitness_function(w, e_r=e_r, cov=cov, rp=rp, alpha = 10, beta = 100):
    w = w.reshape(2, 1)
    return (np.sqrt(w.T@cov@w) + beta*abs(w.T@e_r -rp))[0][0]

In [709]:
def new_population(population, crossover = Heuristic_crossover):
    new_population = []
    intermediate_population=[]
    
    elite = select_elite(population)
    
    for candidate in population:
        c1, c2 = population[np.random.randint(0, len(population))][:], population[np.random.randint(0, len(population))][:]
        ic = candidate +0.1*(c1-c2)
        
        for j in range(len(candidate)):
            ic[j] = np.random.choice([elite[j], ic[j]], size = 1, p=[0.8, 0.2])
        
        ic = np.where(a > 0, a, 0)
        
        if fitness_function(ic) <= fitness_function(candidate):
            new_population.append(ic)
        else: 
            new_population.append(candidate)
    
    return new_population

In [731]:
population_size = 100
n = 2
population = np.array([chromossome(n) for _ in range(population_size)])


for i in range(1000):
    new_population = []
    elite = select_elite(population)
    
    for candidate in population:
        c1, c2 = population[np.random.randint(0, len(population))][:], population[np.random.randint(0, len(population))][:]
        ic = candidate +0.001*(c1-c2)
        
        for j in range(len(candidate)):
            ic[j] = np.random.choice([elite[j], ic[j]], size = 1, p=[0.8, 0.2])
        ic = np.where(ic>=0, ic, 0)
        
        if fitness_function(ic) <= fitness_function(candidate):
            new_population.append(ic)
        else: 
            new_population.append(candidate)
    
        population = np.array([i/sum(i) for i in new_population])

In [739]:
print(fitness_function(select_elite(population)))
print(select_elite(population))

0.016091162026566123
[0.97456287 0.02543713]


In [728]:
from scipy.optimize import minimize

def minimize_vol(target_return, er, cov):
    """
    target_ret -> W
    """
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),)*n
    weights_sum_to_1 = {'type': 'eq',
                        'fun': lambda weights: np.sum(weights) - 1
    }
    return_is_target = {'type': 'eq',
                        'args': (er,),
                        'fun' : lambda weights, er: target_return - weights.T@er
    }
    
    results = minimize(lambda weights, cov: weights.T@cov@weights, init_guess,
                       args=(cov,), method="SLSQP",
                       options={"disp":False},
                       constraints=(return_is_target, weights_sum_to_1),
                       bounds=bounds
                  )
    return results.x

In [735]:
a = minimize_vol(rp, e_r, cov)
a

array([0.66096388, 0.33903612])

In [717]:
e_r

array([[0.00202945],
       [0.00129369]])